In [10]:
import tkinter as tk
import math

In [13]:
state = 'draw'

class Mirror_Room:
    def __init__(self, master):
        self.master = master
        self.master.title("Mirror Room Simulator")
        self.master.geometry("800x600")

        self.controls_frame = tk.Frame(master)
        self.controls_frame.pack(side=tk.LEFT, fill=tk.X)
        
        self.last_point = None
        
        self.idx = 0
        self.visited_idx = -1
        self.mirrors = []
        self.mirror_type = tk.StringVar()
        self.mirror_type.set("flat")
        
        self.launch_point = None
        self.angle_point = None
        
        # self.beam_color = "red"
        # self.beam_width = 2

        self.canvas = tk.Canvas(master, bg="white")
        self.canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        self.canvas.bind("<Button-1>", self.on_click)
        
        # Create the "Reg Mirror" button
        reg_mirror_button = tk.Radiobutton(self.controls_frame, text='Reg Mirror', variable = self.mirror_type, value = "flat")
        reg_mirror_button.pack(padx=5, pady=5)

        # Create the "S Mirror" button
        s_mirror_button = tk.Radiobutton(self.controls_frame, text='S Mirror', variable = self.mirror_type, value = "sphere")
        s_mirror_button.pack(padx=5, pady=5)
        
        # exit button
        exit_button = tk.Button(
            self.controls_frame,
            text='Exit',
            command = self.master.quit
        )
        exit_button.pack(padx=5, pady=5)

        
        
    # def start_simulation(self):
    #     if self.state == 'ready':
    #         self.state = 'launch'
    #     else:
    #         print('not ready')
            
    def on_click(self, event):
        global state
        # print(self.mirror_type.get())
        
        print(state)
        if state == "draw":
            point_click = Point(self.canvas, event.x, event.y)
            if self.last_point is None:
                self.last_point = Point(self.canvas, event.x, event.y)
            else:
                if self.mirror_type.get() == "flat":
                    print('click flat')
                    # self.draw_flat_mirror(event.x, event.y)
                    self.mirrors.append(Mirror(self.canvas, self.last_point, point_click, 0, self.idx, self.mirrors))
                    self.idx += 1
                    # print(self.mirrors)
                    self.last_point = point_click
                
                elif self.mirror_type.get() == "spherical":
                    self.draw_spherical_mirror(event.x, event.y)
        
        elif state == "launch":
            if self.launch_point is None:
                self.launch_point = Point(self.canvas, event.x, event.y)
            else:
                state = 'ready'
                self.angle_point = Point(self.canvas, event.x, event.y)
                self.canvas.create_line(self.launch_point.x, self.launch_point.y, event.x, event.y, fill = 'red')
        
        elif state == 'ready':
            start_sim_button = tk.Button(self.controls_frame, text="Start Simulation",
                                         command = lambda: self.launch_beam(), state = 'normal')
            start_sim_button.pack(side="left", padx=5)
            state = 'sim'
            # launch_beam(self.launch_point.x, self.launch_point.y, event.x, event.y)

            
    def launch_beam(self):
        launch_point = self.launch_point
        end_point = self.angle_point
        print('points:', launch_point.x, launch_point.y, end_point.x, end_point.y)
        # print('test')
        # button.config(state='disabled')
        self.draw_beam(launch_point, end_point)
        
        
    def draw_beam(self, launch_point, end_point):
        closest_mirror = None
        closest_intersection = None
        
        for mirror in self.mirrors:
            mirror_type = mirror.curv
            if mirror.idx != self.visited_idx:
                if mirror_type == 0:
                    intersection = self.cross(launch_point.x, launch_point.y, 
                                              end_point.x, end_point.y, 
                                              mirror.point1.x, mirror.point1.y,
                                              mirror.point2.x, mirror.point2.y)
                if intersection is not None:
                    distance = math.sqrt((end_point.x - intersection[0])**2 + (end_point.y - intersection[1])**2)
                    if closest_intersection is None or distance < closest_intersection:
                        closest_intersection = distance
                        closest_mirror = mirror
        
        print('res:', closest_mirror.idx, closest_intersection)
        self.visited_idx = closest_mirror.idx
        
        if closest_mirror is not None:
            mirror_type = closest_mirror.curv
            if mirror_type == 0:
                self.draw_reflection_flat(launch_point.x, launch_point.y, 
                                          end_point.x, end_point.y, 
                                          closest_mirror.point1, closest_mirror.point2)
                
        self.canvas.create_line(launch_point.x, launch_point.y, end_point.x, end_point.y, fill = 'green')
                    
        
        
        
    def cross(self, x1, y1, x2, y2, x3, y3, x4, y4):
        dot = [0, 0]
        if (y2 - y1 != 0): # a(y)
            q = (x2 - x1) / (y1 - y2)   
            sn = (x3 - x4) + (y3 - y4) * q
            if (sn == 0):
                return None  # c(x) + c(y)*q

            fn = (x3 - x1) + (y3 - y1) * q   # b(x) + b(y)*q
            n = fn / sn

        else:
            if ((y3 - y4) == 0):
                return None  # b(y)
            n = (y3 - y1) / (y3 - y4)   # c(y)/b(y)

        dot[0] = x3 + (x4 - x3) * n  # x3 + (-b(x))*n
        dot[1] = y3 + (y4 - y3) * n  # y3 +(-b(y))*n
        print(dot)
        return dot
        
        
        
    def draw_reflection_flat(self, launch_x, launch_y, end_x, end_y, mirror_start, mirror_end):
        intersection_x, intersection_y = self.cross(launch_x, launch_y, 
                                                    end_x, end_y,
                                                    mirror_start.x, mirror_start.y, 
                                                    mirror_end.x, mirror_end.y)
        if intersection_x is None:
            return None

        # reflect the beam
        reflection_dx = -end_x + intersection_x
        reflection_dy = -end_y + intersection_y
        
        print('refl:', reflection_dx, reflection_dy)
        
        mirror_dx = mirror_end.x - mirror_start.x
        mirror_dy = mirror_end.y - mirror_start.y
        
        print('mirr:', mirror_dx, mirror_dy)
        
        n_dx = -mirror_dy
        n_dy = mirror_dx

        print(n_dx, n_dy)

        reflect_x = reflection_dx - 2 * n_dx * (reflection_dx * n_dx + reflection_dy * n_dy) / (n_dx * n_dx + n_dy * n_dy)
        reflect_y = reflection_dy - 2 * n_dy * (reflection_dx * n_dx + reflection_dy * n_dy) / (n_dx * n_dx + n_dy * n_dy)
        print(reflect_x, reflect_y)
        
        inter = Point(self.canvas, intersection_x, intersection_y)
        reflect = Point(self.canvas, inter.x + reflect_x, inter.y + reflect_y)
        self.canvas.create_line(end_x, end_y, inter.x, inter.y, fill = 'blue')
        self.canvas.create_line(inter.x, inter.y, reflect.x, reflect.y, fill = 'red')
        
        self.launch_point = inter
        self.angle_point = reflect
        print('new_launch', self.launch_point.x, self.launch_point.y, self.angle_point.x, self.angle_point.y)
        # self.draw_beam(inter, reflect)

class Point:
    def __init__(self, canvas, x, y):
        global state
        self.canvas = canvas
        self.x = x
        self.y = y
        if state == 'draw':
            self.id = canvas.create_oval(x-2, y-2, x+2, y+2, fill='black')
        else:
            self.id = canvas.create_oval(x-2, y-2, x+2, y+2, fill='red')
        
        
class Mirror(Mirror_Room):
    def __init__(self, canvas, point1, point2, curv, idx, mirrors):
        # super().__init__(master)
        self.canvas = canvas
        self.point1 = point1
        self.point2 = point2
        self.curv = curv
        self.idx = idx
        self.mirrors = mirrors
        # self.update(idx)
        # print(point1, point2)
        self.create_mirror()
    
    def create_mirror(self):
        global state
        if self.mirrors:
            first_m = self.mirrors[0].point1
            print('FIRST - ', first_m.x, first_m.y)
        
            if ((math.isclose(self.point2.x, first_m.x, abs_tol = 3)) and 
                (math.isclose(self.point2.y, first_m.y, abs_tol = 3))):
                    print(self.point2.x, self.point2.y)
                    self.point2 = first_m
                    print('ready')
                    state = 'launch'
                    print(self.point2.x, self.point2.y)
              
        if self.curv == 0:
            self.body = self.canvas.create_line(self.point1.x, self.point1.y, self.point2.x, self.point2.y)
            self.mirror_type = 'flat'
            print('here')
        else:
            self.body = draw_arc(self.canvas, self.point1, self.point2, self.curv, self.idx)
            self.mirror_type = 'sphere'
    
    def update(self, idx):
        data = Mirror_Room.mirrors[idx]
        # добавить разделение данных
        self.canvas.delete('mirror' + str(idx))
        draw_arc(self.canvas, self.point1, self.point2, self.curvature, idx)

def draw_arc(canvas, start_point, end_point, curvature, idx):
    # Calculate the center point of the arc
    center_x = (start_point[0] + end_point[0]) / 2
    center_y = (start_point[1] + end_point[1]) / 2

    # Calculate the start and end angles of the arc
    start_angle = 180 / math.pi * math.atan2(end_point[1] - center_y, end_point[0] - center_x)
    end_angle = 180 / math.pi * math.atan2(start_point[1] - center_y, start_point[0] - center_x)

    # Draw the arc on the canvas
    canvas.create_arc((center_x - curvature), (center_y - curvature), 
                      (center_x + curvature), (center_y + curvature),
                      start=start_angle, extent=end_angle - start_angle, style='arc', tag='mirror'+str(idx))
    
    

    
if __name__ == "__main__":
    root = tk.Tk()
    app = Mirror_Room(root)
    root.mainloop()

draw
draw
click flat
here
draw
click flat
FIRST -  180 218
here
draw
click flat
FIRST -  180 218
here
draw
click flat
FIRST -  180 218
180 221
ready
180 218
here
launch
launch
ready
points: 240 307 306 232
[386.05286343612335, 141.03083700440527]
[449.340206185567, 69.1134020618557]
[163.83048433048432, 393.5562678062678]
[167.32619114744855, 389.5838736960812]
res: 0 121.17693493582719
[386.05286343612335, 141.03083700440527]
refl: 80.05286343612335 -90.96916299559473
mirr: 257 -96
96 257
120.08804579179997 16.208356435747902
new_launch 386.05286343612335 141.03083700440527 506.1409092279233 157.23919344015317
points: 386.05286343612335 141.03083700440527 506.1409092279233 157.23919344015317
[431.1395499609775, 147.11621445295367]
[-56009.731819697634, -7470.742454757669]
[187.66305357249925, 114.25404394154862]
res: 1 75.6814283392218
[431.1395499609775, 147.11621445295367]
refl: -75.00135926694583 -10.122978987199502
mirr: -70 300
-300 -70
71.73636239282894 24.11582273341461
new_lau